### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [1]:
%pip install langchain

Note: you may need to restart the kernel to use updated packages.


In [5]:
### Open AI API Key and Open Source models--Llama3,Gemma2,mistral--Groq

import os
from dotenv import load_dotenv
load_dotenv()



groq_api_key=os.getenv("groq_api_key")
groq_api_key

'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'

In [12]:
%pip install langchain_groq

In [6]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x1167c35c0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x1167f4680>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
%pip install langchain_core

Note: you may need to restart the kernel to use updated packages.


In [26]:
from langchain_core.messages import HumanMessage,SystemMessage

messages=[
    SystemMessage(content="Translate the following from English to Hindi"),
    HumanMessage(content="How are you?")
]

result=model.invoke(messages)

In [27]:
result

AIMessage(content='आप कैसे हैं? (Aap kaise hain?) \n\n\nThis is the most common and polite way to ask "How are you?" in Hindi. \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 20, 'total_tokens': 56, 'completion_time': 0.065454545, 'prompt_time': 0.00011073, 'queue_time': 0.023009228, 'total_time': 0.065565275}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-09a530b6-16cd-4560-8512-c41d3a2bf7c9-0', usage_metadata={'input_tokens': 20, 'output_tokens': 36, 'total_tokens': 56})

In [28]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(result)

'आप कैसे हैं? (Aap kaise hain?) \n\n\nThis is the most common and polite way to ask "How are you?" in Hindi. \n'

In [29]:
### Using LCEL- chain the components
chain=model|parser
chain.invoke(messages)

'You can say "How are you?" in Hindi in a few ways, depending on the level of formality:\n\n**Formal:**\n\n* **आप कैसे हैं? (Aap kaise hain?)**\n\n**Informal:**\n\n* **तुम कैसे हो? (Tum kaise ho?)**\n* **क्या हाल है? (Kya haal hai?)** \n\nThe most common and versatile option is **आप कैसे हैं?**  (aap kaise hain?).\n\n\n\n'

In [13]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template="Trnaslate the following into {language}:"

prompt=ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)



In [30]:
result=prompt.invoke({"language":"Hindi","text":"good morning"})
result

ChatPromptValue(messages=[SystemMessage(content='Trnaslate the following into Hindi:', additional_kwargs={}, response_metadata={}), HumanMessage(content='good morning', additional_kwargs={}, response_metadata={})])

In [31]:
result.to_messages()

[SystemMessage(content='Trnaslate the following into Hindi:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='good morning', additional_kwargs={}, response_metadata={})]

In [33]:
##Chaining together components with LCEL
chain=prompt|model|parser
chain.invoke({"language":"Hindi","text":"Good morning"})

'The Hindi translation of "Good morning" is **शुभ प्रभात** (Shubh Prabhat). \n'

In [25]:
%pip install streamlit

  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 356.9 kB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.2/731.2 kB 268.9 kB/s eta 0:00:00a 0:00:02
Using cached blinker-1.9.0-py3-none-any.whl (8.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 588.5 kB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 1.2 MB/s eta 0:00:0000:0100:010m
Note: you may need to restart the kernel to use updated packages.
